In [1]:
import random
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

/Users/konglingkun/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
class Smartcoin(object):
    '''
    _Name: Name of Smart token
    _ReservetokenName: Name of Reserve Token
    _CRR: CRR
    _Price: the current Price of smart token
    _Supply: the number of issued smart tokens
    _ReserveBalance: the balance of the reserve pool
    '''
    def __init__(self, name='kennycoin', reservetokenName='aCoin', initCRR=0.5, initPrice=1.0, initIssueNum=100):
        self._Name = name
        self._ReservetokenName = reservetokenName
        self._CRR = initCRR
        self._Price = initPrice
        self._Supply = float(initIssueNum)
        self._ReserveBalance = float(initCRR*initIssueNum)

        # _initPrice is used to set the init Price of Market
        self._initPrice = initPrice

    def printInfo(self):
        print '---------'
        print 'NAME:', self._Name, '| RESERVE NAME:', self._ReservetokenName, 'CRR:', self._CRR
        print 'PRICE:',self._Price
        print 'SUPPLY:', self._Supply, '| RESERVE BALANCE:', self._ReserveBalance

    def saveInfo(self, fw):
        fw.write('---------\n')
        fw.write('NAME: '+str(self._Name)+' | RESERVE NAME: '+str(self._ReservetokenName) + ' | CRR: '+str(self._CRR)+'\n')
        fw.write('PRICE: '+str(self._Price))
        fw.write('SUPPLY: '+str(self._Supply)+' | RESERVE BALANCE: '+str(self._ReserveBalance)+'\n')
        
    def getInitPrice(self):
        return self._initPrice

    def updatePrice(self, reserveBalance, supply, CRR):
        newPrice = reserveBalance / (supply * CRR)
        return newPrice
    
    def setCRR(self, newCRR = 0.5):
        oldCRR = self._CRR
        self._CRR = newCRR
        print 'CRR', oldCRR, '->', newCRR

    def getPrice(self):
        return self._Price

    def purchasing(self, convertIntoNum=0):
        # E.g., ETH be convert into BNT, convertInto Num
        issuedtokenNum = round( self._Supply * (((self._ReserveBalance + convertIntoNum)/self._ReserveBalance)**(self._CRR) - 1) )
        self._Supply = self._Supply + issuedtokenNum
        self._ReserveBalance = self._ReserveBalance + convertIntoNum
        # Update the price after purchasing, e.g. ETH is converted into BNT (cust uses eth to buy bnt)
        oldPrice = self._Price
        self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
        increaseRatio = (self._Price - oldPrice)/oldPrice
        return int(issuedtokenNum)

    def destroying(self, convertOutNum=0):
        # E.g., BNT be converted out to ETH, convertOutNum is BNT's num
        destroyedtokenNum = convertOutNum
        reserveReceivedNum = round(self._ReserveBalance*(1 - ((self._Supply - convertOutNum) / self._Supply) ** (1/self._CRR)))
        self._Supply = self._Supply - destroyedtokenNum
        self._ReserveBalance = self._ReserveBalance - reserveReceivedNum
        # Update the price after destroying, e.g. BNT is converted into ETH (cust sells bnt to get eth)
        oldPrice = self._Price
        self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
        decreaseRatio = (oldPrice-self._Price)/oldPrice
        return int(reserveReceivedNum)

# def smarttoken_main():
#     ''' 
#     This part is consistent with the example offered by white paper, 
#         which shows our codes realize the idea of Bancor according to its white paper.
#     '''
#     BNTCoin = Smartcoin(name='BNT',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)
#     BNTCoin.printInfo()
#     print BNTCoin.destroying(300)
#     print BNTCoin.purchasing(300)
#     print BNTCoin.purchasing(700)
#     BNTCoin.printInfo()
#     print BNTCoin.destroying(1302)
#     BNTCoin.printInfo()
#     print BNTCoin.purchasing(100)
#     BNTCoin.printInfo()

# if __name__ == '__main__':
#     smarttoken_main()

In [3]:
class Customer(object):
    '''
    _smartToken: class of the token customers want to buy or sell -- SmartToken()
    _market: instance of the market customers buy or sell 
    _tokenBalance: number of smart tokens a customer has
    _reserveBalance: number of reserve tokens a customer has
    _valuation: how much money customers are willing to finish the transaction
    _originalCash: original money cust has
    _cash: net profit or loss to date
    _timeslot: the time slot when customers want to make transactions, 
               and in every timeslot, we should set customers' time slot like: XXX.SetTimeSlot(time)
    '''
    def __init__(self, smartToken, market ,tokenBalance = 0, reserveBalance = 0, valuation = 0.0):
        self._smartToken = smartToken
        self._market = market
        self._tokenBalance = tokenBalance
        self._reserveBalance = reserveBalance
        self._originalCash = float(self._reserveBalance + self._tokenBalance * self._smartToken.getPrice())
        self._cash = 0.0
        self._valuation = valuation
        self._timeslot = None

        self._BUY = 1
        self._SELL = -1
        self._ERROR = -2
        self._TransactionFailed = -1

    def printInfo(self):
        currentCash = float(self._smartToken.getPrice() * self._tokenBalance + self._reserveBalance)
        self._cash = currentCash - self._originalCash
        print '------'
        print 'smartToken Name:', self._smartToken._Name, '| valuation:', self._valuation
        print 'reserveBalance:', self._reserveBalance, '| tokenBalance:', self._tokenBalance

    # def SetTimeSlot(self, timeslot):
    #     self._timeslot = timeslot

    def getReserveBalance(self):
        return self._reserveBalance

    def changeReserveBalance(self, changeValue):
        self._reserveBalance = self._reserveBalance + changeValue

    def getTokenBalance(self):
        return self._tokenBalance

    def changeTokenBalance(self, changeValue):
        self._tokenBalance = self._tokenBalance + changeValue

    def getCash(self):
        currentCash = float(self._smartToken.getPrice() * self._tokenBalance + self._reserveBalance)
        self._cash = currentCash - self._originalCash
        return self._cash

    def getValuation(self):
        return self._valuation

    # change the valuation, if customer has a new valuation, he will generate a new transaction request, 
    # and market will give responce for this request
    def changeValuation(self, newValuation):
        self._valuation = newValuation
        '''
        Cancel the previous order in market's orderlist.
        If the previous order succeeds in the market, this function call will cancel nothing.
        '''
        self._market.cancelOrder(self)
        '''
        What getCurrentPrice() returns in Bancor market should be different with the real time price of Smart Token. 
        This is because since every time slot many customers come into the market simultaneously, 
            what they see is the final price at the end of previous time slot.
        '''
        if self._valuation > self._market.getCurrentPrice() and self._reserveBalance > 0:
            # XXX issues a buy order 
            self._market.buy(self, self._reserveBalance)
        elif self._valuation < self._market.getCurrentPrice() and self._tokenBalance > 0:
            # XXX issue a sell order
            self._market.sell(self, self._tokenBalance)
        else:
            # nothing to do
            pass

# def cust_main():
    
#     KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1.0, initIssueNum=300000)
    
#     # test for Bancor Market
#     market1 = BancorMarket(smartToken = KennyCoin)
#     Alice = Customer(smartToken=KennyCoin, market=market1, tokenBalance=200, reserveBalance=100)
#     market1.sychronize(0)
#     Alice.printInfo()
#     Alice.changeValuation(1.5)
#     Alice.printInfo()
#     KennyCoin.printInfo()
#     Alice.changeValuation(0.9)
#     Alice.printInfo()
#     KennyCoin.printInfo()
#     print market1.getCanceledTransactionNum() , 'being canceled.'

#     # test for Classic Market
#     market2 = ClassicMarket(smartToken = KennyCoin)
#     Alice = Customer(smartToken=KennyCoin, market=market2, tokenBalance=200, reserveBalance=100)
#     Bob = Customer(smartToken=KennyCoin, market=market2, tokenBalance=305, reserveBalance=333)
#     market2.sychronize(0)
#     Alice.changeValuation(1.5)
#     Alice.printInfo()
#     Bob.changeValuation(1.6)
#     Bob.changeValuation(0.7)
#     Alice.printInfo()
#     Bob.printInfo()
#     Alice.changeValuation(1.4)
#     print market2.getCanceledTransactionNum(), 'being canceled.'
#     print market2.getTotallyFailedTransactionNum(), 'Totally failed.'

# if __name__ == '__main__':
#     cust_main()
    

In [4]:
class BancorMarket(object):
    def __init__(self, smartToken):
        self._smartToken = smartToken
        # Order format: [cust, transactionValue, buy_or_sell]
        self._OrderList = []
        self._CurrentPrice = smartToken.getInitPrice()
        self._timeList = []
        self._time = 0

        self._BUY = 1
        self._SELL = -1
        self._ERROR = -2

        # parameters for plotting: txNum and canceledTxNum are reset to 0 in the beginning of every time slot:
        self._transactionNum = 0
        self._canceledTransactionNum = 0

    '''
    To tell market new time slot is coming, 
        and the currentPrice market offers to customers should be different from the previous one.
    '''
    def sychronize(self, timeSlot):
        self._time = timeSlot
        self._transactionNum = 0
        self._canceledTransactionNum = 0

    # what getCurrentPrice() returns in Bancor market should be different with the real time price of Smart Token. 
    # This is because since every time slot many customers come into the market simultaneously, 
    # what they see is the final price at the end of previous time slot
    def getCurrentPrice(self):
        if self._time in self._timeList:
            return self._CurrentPrice
        else:
            self._timeList.append(self._time)
            self._CurrentPrice = self._smartToken.getPrice()
            return self._CurrentPrice

    def cancelOrder(self, cust):
        # Order format: [cust, transactionValue, buy_or_sell]
        for s in range(len(self._OrderList)):
            if self._OrderList[s][0] is cust:
                self._canceledTransactionNum = self._canceledTransactionNum + 1
                self._OrderList.pop(s)
                break
    '''
    Becuase in Bancor Market, after every transaction, the price of smart token will be changed.
    Update the orderlist by recursion after every time transaction being made is too time-comsuming.
    Here, we just scan the orderlist once if the price being changed. 
        E.g., every time the price of token changes, we will sequence through the orderlist to see whether some orders can be satisfied.
    And this method offers almost same accuracy with recursion function.
    '''
    def updateOrderList(self):
        s = 0
        while s < len(self._OrderList):
            if ((self._OrderList[s][2] == self._BUY) and (self._OrderList[s][0].getValuation()>=self._smartToken.getPrice()) ):
                receivedSmartTokens = self._smartToken.purchasing(self._OrderList[s][1])
                self._OrderList[s][0].changeReserveBalance(-self._OrderList[s][1])
                self._OrderList[s][0].changeTokenBalance(receivedSmartTokens)
                self._OrderList.pop(s)
                s = s - 1
            elif ((self._OrderList[s][2] == self._SELL) and (self._OrderList[s][0].getValuation()<=self._smartToken.getPrice()) ):
                receivedReserveTokens = self._smartToken.destroying(self._OrderList[s][1])
                self._OrderList[s][0].changeReserveBalance(receivedReserveTokens)
                self._OrderList[s][0].changeTokenBalance(-self._OrderList[s][1])
                self._OrderList.pop(s)
                s = s - 1
            else:
                # nothing to do
                pass
            s = s + 1
    '''
    use #Transaction_Value reserveTokens to buy smartTokens -> smartToken price increase
    call smartTokens.purchasing() function
    '''
    def buy(self, cust, Transaction_Value):
        self._transactionNum = self._transactionNum + 1
        if Transaction_Value < 0:
            print '** ERROR, cannot buy with negative number of reserveToken'
        if not isinstance(Transaction_Value,int):
            print '** ERROR, should use integer number of reserveTokens to buy', Transaction_Value
        if cust.getValuation() >= self._smartToken.getPrice():
            receivedSmartTokens = self._smartToken.purchasing(Transaction_Value)
            cust.changeReserveBalance(-Transaction_Value)
            cust.changeTokenBalance(receivedSmartTokens)
            '''
            Since the price of smart token is changed, maybe some transaction requests in orderlist is now acceptable, 
                we need to update the OrderList Now.
            '''
            self.updateOrderList()
        else:
            # add buy order into orderlist
            self._OrderList.append([cust, Transaction_Value, self._BUY])
            
        
    '''
    sell #Transaction_Value smartTokens to get reserveTokens -> smartToken price decrease
    call smartTokens.destroying() function
    '''
    def sell(self, cust, Transaction_Value):
        self._transactionNum = self._transactionNum + 1
        if Transaction_Value < 0:
            print '** ERROR, cannot sell negative number of smartToken'
        if not isinstance(Transaction_Value,int):
            print '** ERROR, should use integer number of smartTokens to sell', Transaction_Value
        if cust.getValuation() <= self._smartToken.getPrice():
            receivedReserveTokens = self._smartToken.destroying(Transaction_Value)
            cust.changeReserveBalance(receivedReserveTokens)
            cust.changeTokenBalance(-Transaction_Value)
            '''
            Since the price of smart token is changed, maybe some transaction requests in orderlist is now acceptable, 
                we need to update the OrderList Now.
            '''
            self.updateOrderList()
        else:
            # add sell order into orderlist
            self._OrderList.append([cust, Transaction_Value, self._SELL])

    # functions for plotting
    def getTransactionNum(self):
        return self._transactionNum
        
    def getCanceledTransactionNum(self):
        return self._canceledTransactionNum



'''
Custmers give their orders to Classic Market. Market will process these orders.
Once customers' orders come into market, market will update the order list.
And if customers want to change their valuation, their old orders will be canceled, 
  which means the order is failed (or partially failed -- 
  e.g. Customer XXX generates a sell order of 100 tokens, he successfully sells 50, but still 50 remaining in orderlist)
In Classic Market, the price of smart token will not change, but the customers' valuations will change.
'''
class ClassicMarket(object):
    def __init__(self, smartToken):
        self._smartToken = smartToken
        # Order format: [cust, transactionValue, buy_or_sell]
        self._OrderList = []
        self._CurrentPrice = smartToken.getInitPrice()

        self._BUY = 1
        self._SELL = -1
        self._ERROR = -2

        # parameters for plotting:
        self._transactionNum = 0
        self._canceledTransactionNum = 0
        self._totallyFailedTransactionNum = 0
        self._ChangedOrderList = []


    '''
    Reset plotting parameters.
    '''
    def sychronize(self, timeSlot = 0):
        self._transactionNum = 0
        self._canceledTransactionNum = 0
        self._totallyFailedTransactionNum = 0

    def getCurrentPrice(self):
        return self._CurrentPrice

    def cancelOrder(self, cust):
        # Order format: [cust, transactionValue, buy_or_sell]
        for s in range(len(self._OrderList)):
            if self._OrderList[s][0] is cust:
                self._canceledTransactionNum = self._canceledTransactionNum + 1
                if cust not in self._ChangedOrderList:
                    self._totallyFailedTransactionNum = self._totallyFailedTransactionNum + 1
                else:
                    # Since the order will be canceled, the customer should be removed from ChangedOrderList.
                    self._ChangedOrderList.pop(self._ChangedOrderList.index(cust))
                self._OrderList.pop(s)
                break

    def updateOrderList(self, newOrder):
        cust = newOrder[0]
        custValuation = cust.getValuation()
        transactionValue =  newOrder[1]
        buy_or_sell = newOrder[2]

        # If the orderlist is empty, just add the new order into list
        if len(self._OrderList) == 0:
            self._OrderList.append(newOrder)
            return
        '''
        Buyer comes into the market, scan the market seller, find sellers who offers the valuation smaller than his valuation:
            If none of the sellers offers smaller valuation, push new order into list, return
            If there do exist some sellers, save [seller's valuation, seller, seller's index in orderList] into a seller list, 
            and sorted by sellers' valuations from small to large. 
            Then, buyer will try to make transaction with the seller in list one by one:
            Loop:
                if the buyer's demand can be satisfied by seller:
                    update the buyer's info in customer class,
                    update the seller's info in customer class, 
                    update the seller's info in order list. 
                    return
                if the buyer's demand can not be fully satisfied by seller:
                    update the buyer's info in customer class,
                    decrease buyer's demand
                    update the seller's info in customer class,
                    pop the seller out of order list
                    update other sellers in seller list's index info since one seller is poped out of orderlist 
            if buyer's demand still > 0:
                push buyer's remaining demand into order list
            else:
                just return, since the buyer's demand is satisfied by transaction

        Same thing goes with seller. But for seller, the buyer list should be sorted from high valuation to low valuation
        '''
        sellerList = []
        buyerList = []
        if buy_or_sell == self._BUY:
            for s in range(len(self._OrderList)):
                if self._OrderList[s][2] == self._SELL and custValuation >= self._OrderList[s][0].getValuation():
                    sellerValuation = self._OrderList[s][0].getValuation()
                    seller = self._OrderList[s][0]
                    # sellerList format: [valuation, seller, index of seller in orderList]
                    sellerList.append([sellerValuation,seller,s])

            if len(sellerList) == 0:
                self._OrderList.append(newOrder)
                return
            else:
                sorted(sellerList, key=lambda sellerOrders: sellerOrders[0]) # sorted from low to high

            for t in range(len(sellerList)):
                sellerValuation = sellerList[t][0]
                seller = sellerList[t][1]
                indexInOrderList = sellerList[t][2]
                sellerMaxCouldSatisfy = int(self._OrderList[indexInOrderList][1] * sellerValuation)
                if transactionValue < sellerMaxCouldSatisfy:
                    cust.changeReserveBalance(-transactionValue)
                    cust.changeTokenBalance(int(transactionValue/sellerValuation))
                    seller.changeReserveBalance(transactionValue)
                    seller.changeTokenBalance(-int(transactionValue/sellerValuation))
                    self._OrderList[indexInOrderList][1] -= int(transactionValue/sellerValuation)
                    if seller not in self._ChangedOrderList:
                        # to count whether cust's order is totally failed
                        self._ChangedOrderList.append(seller)
                    break # cust's order is satisfied
                else: 
                # seller cannot satisfy buyer's need, transactionValue >= sellerMaxCouldSatisfy
                    cust.changeReserveBalance(-sellerMaxCouldSatisfy)
                    cust.changeTokenBalance(self._OrderList[indexInOrderList][1])
                    transactionValue -= sellerMaxCouldSatisfy
                    seller.changeReserveBalance(sellerMaxCouldSatisfy)
                    seller.changeTokenBalance(-self._OrderList[indexInOrderList][1])
                    self._OrderList.pop(indexInOrderList)
                    for item in sellerList:
                        if item[2] > indexInOrderList:
                            # update index info, since one seller is poped out from orderList
                            item[2] -= 1

            if transactionValue < 0:
                print '** Error, buyer buys too much in market class'
            elif transactionValue == 0:
                return
            else:
                # transactionValue > 0
                newOrder[1] = transactionValue
                self._OrderList.append(newOrder)
        else:
            # buy_or_sell == self._SELL
            for s in range(len(self._OrderList)):
                if self._OrderList[s][2] == self._BUY and custValuation <= self._OrderList[s][0].getValuation():
                    buyerValuation = self._OrderList[s][0].getValuation()
                    buyer = self._OrderList[s][0]
                    # buyerList format: [valuation, buyer, index of buyer in orderList]
                    buyerList.append([buyerValuation,buyer,s])

            if len(buyerList) == 0:
                self._OrderList.append(newOrder)
                return
            else:
                # sorted from high to low
                sorted(sellerList, key=lambda sellerOrders: sellerOrders[0], reverse=True) 

            for t in range(len(buyerList)):
                buyerValuation = buyerList[t][0]
                buyer = buyerList[t][1]
                indexInOrderList = buyerList[t][2]
                buyerMaxCouldBuy = int(self._OrderList[indexInOrderList][1] / buyerValuation)
                if transactionValue < buyerMaxCouldBuy:
                    cust.changeReserveBalance(int(transactionValue * buyerValuation))
                    cust.changeTokenBalance(-transactionValue)
                    buyer.changeReserveBalance(-int(transactionValue * buyerValuation))
                    buyer.changeTokenBalance(transactionValue)
                    self._OrderList[indexInOrderList][1] -= int(transactionValue * buyerValuation)
                    if buyer not in self._ChangedOrderList:
                        # to count whether cust's order is totally failed
                        self._ChangedOrderList.append(buyer)
                    break # cust's order is satisfied
                else:
                    # buyer cannot satisfy the seller's need, transactionValue >= buyerMaxCouldBuy
                    cust.changeReserveBalance(self._OrderList[indexInOrderList][1])
                    cust.changeTokenBalance(-buyerMaxCouldBuy)
                    transactionValue -= buyerMaxCouldBuy
                    buyer.changeReserveBalance(-self._OrderList[indexInOrderList][1])
                    buyer.changeTokenBalance(buyerMaxCouldBuy)
                    self._OrderList.pop(indexInOrderList)
                    for item in buyerList:
                        if item[2] > indexInOrderList:
                            # update index info, since one buyer is poped out from orderList
                            item[2] -= 1 

            if transactionValue < 0:
                print '** Error, Seller sells too much in market class'
            elif transactionValue == 0:
                return
            else:
                # transactionValue > 0
                newOrder[1] = transactionValue
                self._OrderList.append(newOrder)

    '''
    use #Transaction_Value reserveTokens to buy smartTokens -> smartToken price increase
    call smartTokens.purchasing() function
    '''
    def buy(self, cust, Transaction_Value):
        self.updateOrderList([cust, Transaction_Value, self._BUY])
        self._transactionNum = self._transactionNum + 1
        
    '''
    sell #Transaction_Value smartTokens to get reserveTokens -> smartToken price decrease
    call smartTokens.destroying() function
    '''
    def sell(self, cust, Transaction_Value):
        self.updateOrderList([cust, Transaction_Value, self._SELL])
        self._transactionNum = self._transactionNum + 1

    # functions for plotting
    def getTransactionNum(self):
        return self._transactionNum
        
    def getCanceledTransactionNum(self):
        return self._canceledTransactionNum

    def getTotallyFailedTransactionNum(self):
        return self._totallyFailedTransactionNum

In [ ]:
# Main function call for Bancor Market Simulating
# the Bancor Market has to be sychronized in every different time slot
def sychronizeMarket(market, timeSlot):
    market.sychronize(timeSlot)

# issue a new smart token
initIssue = 3000000
CRR = 0.2
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=initIssue)

# create two different markets
MyBancorMarket = BancorMarket(smartToken = KennyCoin)

TimeSlotNum = 1000
bouncingInterval = 200
bouncingRange = 10.0
custNum = 2000
sigma = 0.1

# the seeds of pseudo-random numbers
mySeeds = [0,1,2,3,4]

for mySeed in mySeeds:
    np.random.seed(mySeed)
    myfw = open('Result/Bancor/T-'+str(TimeSlotNum)+'BI-'+str(bouncingInterval)+
        'BG-'+str(bouncingRange)+'CN-'+str(custNum)+'Sig-'+str(sigma)+'Seed-'+str(mySeed)+'.txt', 'w')

    '''
    First of all, we initialize the customer's tokenBalance and reserveBalance 
    by Gaussian distributed random number (mu = 200, sigma = 0.1)
    '''
    custInitReserveBalance_mu = 200
    custInitTokenBalance_mu = 200
    custInitReserveBalance_list = np.random.normal(custInitReserveBalance_mu, 0.1, custNum) # 0.5 is sigma
    custInitTokenBalance_list = np.random.normal(custInitTokenBalance_mu, 0.1, custNum) # 0.5 is sigma

    if sum(custInitTokenBalance_list) > (initIssue * (1 - CRR)):
        print 'ERROR, too many init smart tokens holding by customers.'
        sys.exit(0)

    custList = []
    # here we name single customer as Joe. And every customer is initialized with 
    # random value of token balance as well as reserve balance.
    for i in range(custNum):
        Joe = Customer(smartToken = KennyCoin, market = MyBancorMarket, 
                        tokenBalance = int(custInitTokenBalance_list[i]), 
                        reserveBalance = int(custInitReserveBalance_list[i]))
        custList.append(Joe)

    # cashTracker records custmers' cash
    # cashTracker = []
    # priceTracker records the change of the smart token's price in Bancor market
    priceTracker = []
    # transaction tracker records the transations' number in each time slot
    txTracker = []
    # canceled transaction tracker records the canceled transactions' number in each time slot
    canceledTxTracker = []

    for j in range(TimeSlotNum):
        # Sychronize the market
        sychronizeMarket(MyBancorMarket, j)

        # we assume that in every time slot, all customers change their valuation
        currentMarketPrice = MyBancorMarket.getCurrentPrice()
        if (j > 0) and (j % bouncingInterval == 0):
            ''' 
            We assume the valuation_mu is generated by random, which denotes the mean valuation
            of customers when the good or bad news comes into market on a certain time slot,
            which is divided by bouncing interval.
            '''
            valuation_mu = random.uniform(currentMarketPrice/bouncingRange, currentMarketPrice*bouncingRange)
        else:
            valuation_mu = currentMarketPrice

        custValuation_list = np.random.normal(valuation_mu, sigma, custNum)
        for i in range(custNum):
            if custValuation_list[i] < 0:
                # Customer does not want to sell their token in free. 
                # Here we give them a small valuation when valuation < 0
                custList[i].changeValuation(0.001*currentMarketPrice)
            else:
                custList[i].changeValuation(custValuation_list[i])

        priceTracker.append(KennyCoin.getPrice())
        txTracker.append(MyBancorMarket.getTransactionNum())
        canceledTxTracker.append(MyBancorMarket.getCanceledTransactionNum())

        # show some information in terminal
        print ('In time slot:'+str(j)+' | '+str(MyBancorMarket.getTransactionNum())+
            ' happens. And '+str(MyBancorMarket.getCanceledTransactionNum())+' transactions are canceled.')
        myfw.write(str(j)+'\t'+str(MyBancorMarket.getTransactionNum())+'\t'+
                    str(MyBancorMarket.getCanceledTransactionNum())+'\n')
    myfw.close()

    '''Plotting'''

    # Figure about price changing
    pricePlot = []
    myX_P = []
    for j in range(TimeSlotNum):
        pricePlot.append(priceTracker[j])
        myX_P.append(j)
        j = j + 1
    x_P = np.asarray(myX_P)
    y_P = np.asarray(pricePlot)
    plt.plot(x_P, y_P, 'o-',color = 'navy', alpha = 0.8)
    plt.title('Price Change For Bancor Market',fontsize = 25)
    plt.xlabel('Time Slot #',fontsize = 15)
    plt.ylabel('Price of Smart Token', fontsize = 15)
    plt.savefig('Figures/Bancor/Price-Seed-'+str(mySeed)+'.png', bbox_inches='tight')
    plt.close()

    # Figure about transactions
    txPlot = []
    myX_T = []
    for j in range(TimeSlotNum):
        txPlot.append(txTracker[j])
        myX_T.append(j)
        j = j + 1
    x_T = np.asarray(myX_T)
    y_T = np.asarray(txPlot)
    plt.plot(x_T, y_T, 'o-',color = 'navy', alpha = 0.8)
    plt.title('Transaction Num For Bancor Market',fontsize = 25)
    plt.xlabel('Time Slot #',fontsize = 15)
    plt.ylabel('Transaction #', fontsize = 15)
    plt.savefig('Figures/Bancor/Transactions-Seed-'+str(mySeed)+'.png', bbox_inches='tight')
    plt.close()

    # Figure about canceled transactions
    canceledTxPlot = []
    myX_C = []
    for j in range(TimeSlotNum):
        canceledTxPlot.append(canceledTxTracker[j])
        myX_C.append(j)
        j = j + 1
    x_C = np.asarray(myX_C)
    y_C = np.asarray(canceledTxPlot)
    plt.plot(x_C, y_C, 'o-',color = 'navy', alpha = 0.8)
    plt.title('Canceled Transaction Num For Bancor Market',fontsize = 25)
    plt.xlabel('Time Slot #',fontsize = 15)
    plt.ylabel('Canceled Transaction #', fontsize = 15)
    plt.savefig('Figures/Bancor/CanceledTx-Seed-'+str(mySeed)+'.png', bbox_inches='tight')
    plt.close()

    fw_trax = open('Result/Bancor/Tx_T-'+str(TimeSlotNum)+'BI-'+str(bouncingInterval)+
        'BG-'+str(bouncingRange)+'CN-'+str(custNum)+'Sig-'+str(sigma)+'Seed-'+str(mySeed)+'.txt', 'w')
    fw_trax.write('All_Tx:'+'\t'+str(sum(txTracker))+'\tCanceled:'+'\t'+str(sum(canceledTxTracker)))
    fw_trax.close()



In [5]:
# Main function call for Classic Market simulating
# the Classic Market has to be sychronized in every different time slot
def sychronizeMarket(market, timeSlot):
    market.sychronize(timeSlot)

# issue a new smart token
initIssue = 3000000
CRR = 0.2
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=initIssue)

# create two different markets
MyClassicMarket = ClassicMarket(smartToken = KennyCoin)

TimeSlotNum = 1000
bouncingInterval = 200
bouncingRange = 10.0
custNum = 2000
sigma = 0.1

# the seeds of pseudo-random numbers
mySeeds = [0,1,2,3,4]

for mySeed in mySeeds:
    np.random.seed(mySeed)
    myfw = open('Result/Classic/T-'+str(TimeSlotNum)+'BI-'+str(bouncingInterval)+
        'BG-'+str(bouncingRange)+'CN-'+str(custNum)+'Sig-'+str(sigma)+'Seed-'+str(mySeed)+'.txt', 'w')

    '''
    First of all, we initialize the customer's tokenBalance and reserveBalance 
    by Gaussian distributed random number (mu = 200, sigma = 0.1)
    '''
    custInitReserveBalance_mu = 200
    custInitTokenBalance_mu = 200
    custInitReserveBalance_list = np.random.normal(custInitReserveBalance_mu, 0.1, custNum) # 0.5 is sigma
    custInitTokenBalance_list = np.random.normal(custInitTokenBalance_mu, 0.1, custNum) # 0.5 is sigma

    if sum(custInitTokenBalance_list) > (initIssue * (1 - CRR)):
        print 'ERROR, too many init smart tokens holding by customers.'
        sys.exit(0)

    custList = []
    # here we name single customer as Joe. And every customer is initialized with 
    # random value of token balance as well as reserve balance.
    for i in range(custNum):
        Joe = Customer(smartToken = KennyCoin, market = MyClassicMarket, 
                        tokenBalance = int(custInitTokenBalance_list[i]), 
                        reserveBalance = int(custInitReserveBalance_list[i]))
        custList.append(Joe)

    # cashTracker records custmers' cash
    # cashTracker = []
    # transaction tracker records the transations' number in each time slot
    txTracker = []
    # canceled transaction tracker records the canceled transactions' number in each time slot
    canceledTxTracker = []
    # failed transaction tracker records the totally failed transactions' number in each time slot
    failedTxTracker = []

    for j in range(TimeSlotNum):

        # Sychronize the market
        sychronizeMarket(MyClassicMarket, j)

        # we assume that in every time slot, all customers change their valuation
        currentMarketPrice = MyClassicMarket.getCurrentPrice()
        if (j > 0) and (j % bouncingInterval == 0):
            ''' 
            We assume the valuation_mu is generated by random, which denotes the mean valuation
            of customers when the good or bad news comes into market on a certain time slot,
            which is divided by bouncing interval.
            '''
            valuation_mu = random.uniform(currentMarketPrice/bouncingRange, currentMarketPrice*bouncingRange)
        else:
            valuation_mu = currentMarketPrice

        custValuation_list = np.random.normal(valuation_mu, sigma, custNum)
        for i in range(custNum):
            if custValuation_list[i] < 0:
                # Customer does not want to sell their token in free. 
                # Here we give them a small valuation when valuation < 0
                custList[i].changeValuation(0.001*currentMarketPrice)
            else:
                custList[i].changeValuation(custValuation_list[i])

        txTracker.append(MyClassicMarket.getTransactionNum())
        canceledTxTracker.append(MyClassicMarket.getCanceledTransactionNum())
        failedTxTracker.append(MyClassicMarket.getTotallyFailedTransactionNum())

        # show some information in terminal
        print ('In time slot:'+str(j)+' | '+str(MyClassicMarket.getTransactionNum())+
            ' happens. And '+str(MyClassicMarket.getCanceledTransactionNum())+' transactions are canceled. | And '+
            str(MyClassicMarket.getTotallyFailedTransactionNum())+' totally failed.')
        myfw.write(str(j)+'\t'+str(MyClassicMarket.getTransactionNum())+'\t'+
                    str(MyClassicMarket.getCanceledTransactionNum())+'\t'+
                    str(MyClassicMarket.getTotallyFailedTransactionNum())+'\n')
    myfw.close()
    
    '''Plotting'''

    # Figure about transactions
    txPlot = []
    myX_T = []
    for j in range(TimeSlotNum):
        txPlot.append(txTracker[j])
        myX_T.append(j)
        j = j + 1
    x_T = np.asarray(myX_T)
    y_T = np.asarray(txPlot)
    plt.plot(x_T, y_T, 'o-',color = 'navy', alpha = 0.8)
    plt.title('Transaction Num For Classic Market',fontsize = 25)
    plt.xlabel('Time Slot #',fontsize = 15)
    plt.ylabel('Transaction #', fontsize = 15)
    plt.savefig('Figures/Classic/Transactions-Seed-'+str(mySeed)+'.png', bbox_inches='tight')
    plt.close()

    # Figure about canceled transactions
    canceledTxPlot = []
    myX_C = []
    for j in range(TimeSlotNum):
        canceledTxPlot.append(canceledTxTracker[j])
        myX_C.append(j)
        j = j + 1
    x_C = np.asarray(myX_C)
    y_C = np.asarray(canceledTxPlot)
    plt.plot(x_C, y_C, 'o-',color = 'navy', alpha = 0.8)
    plt.title('Canceled Transaction Num For Classic Market',fontsize = 25)
    plt.xlabel('Time Slot #',fontsize = 15)
    plt.ylabel('Canceled Transaction #', fontsize = 15)
    plt.savefig('Figures/Classic/CanceledTx-Seed-'+str(mySeed)+'.png', bbox_inches='tight')
    plt.close()

    # Figure about failed transactions
    failedTxPlot = []
    myX_F = []
    for j in range(TimeSlotNum):
        failedTxPlot.append(failedTxTracker[j])
        myX_F.append(j)
        j = j + 1
    x_F = np.asarray(myX_F)
    y_F = np.asarray(failedTxPlot)
    plt.plot(x_F, y_F, 'o-',color = 'navy', alpha = 0.8)
    plt.title('Failed Transaction Num For Classic Market',fontsize = 25)
    plt.xlabel('Time Slot #',fontsize = 15)
    plt.ylabel('Failed Transaction #', fontsize = 15)
    plt.savefig('Figures/Classic/FailedTx-Seed-'+str(mySeed)+'.png', bbox_inches='tight')
    plt.close()

    fw_trax = open('Result/Classic/Tx_T-'+str(TimeSlotNum)+'BI-'+str(bouncingInterval)+
        'BG-'+str(bouncingRange)+'CN-'+str(custNum)+'Sig-'+str(sigma)+'Seed-'+str(mySeed)+'.txt', 'w')
    fw_trax.write('All_Tx:'+'\t'+str(sum(txTracker))+'\tCanceled:'+'\t'+str(sum(canceledTxTracker))
        +'\tTotally_failed:'+'\t'+str(sum(failedTxTracker)))
    fw_trax.close()


In time slot:0 | 2000 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:1 | 984 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:2 | 1045 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:3 | 977 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:4 | 1020 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:5 | 994 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:6 | 1009 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:7 | 1018 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:8 | 985 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:9 | 1012 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:10 | 1005 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:11 | 1002 happens. And

In time slot:96 | 1006 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:97 | 1003 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:98 | 977 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:99 | 956 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:100 | 978 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:101 | 1001 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:102 | 989 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:103 | 929 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:104 | 967 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:105 | 974 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:106 | 945 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:107 | 958

In time slot:192 | 852 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:193 | 810 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:194 | 811 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:195 | 816 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:196 | 843 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:197 | 811 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:198 | 843 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:199 | 810 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:200 | 832 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:201 | 591 happens. And 7 transactions are canceled. | And 0 totally failed.
In time slot:202 | 598 happens. And 8 transactions are canceled. | And 1 totally failed.
In time slot:203 | 61

In time slot:289 | 390 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:290 | 410 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:291 | 406 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:292 | 425 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:293 | 428 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:294 | 416 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:295 | 395 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:296 | 397 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:297 | 419 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:298 | 388 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:299 | 393 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:300 | 38

In time slot:385 | 289 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:386 | 306 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:387 | 307 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:388 | 290 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:389 | 298 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:390 | 290 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:391 | 288 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:392 | 283 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:393 | 296 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:394 | 309 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:395 | 308 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:396 | 28

...

In time slot:626 | 85 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:627 | 92 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:628 | 89 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:629 | 86 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:630 | 72 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:631 | 81 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:632 | 77 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:633 | 86 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:634 | 77 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:635 | 88 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:636 | 81 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:637 | 96 happens. A

In time slot:728 | 59 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:729 | 55 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:730 | 49 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:731 | 52 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:732 | 50 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:733 | 56 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:734 | 52 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:735 | 52 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:736 | 58 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:737 | 52 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:738 | 52 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:739 | 55 happens. A

In time slot:831 | 26 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:832 | 32 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:833 | 40 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:834 | 40 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:835 | 44 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:836 | 41 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:837 | 29 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:838 | 34 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:839 | 34 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:840 | 37 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:841 | 37 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:842 | 30 happens. A

In time slot:934 | 29 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:935 | 26 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:936 | 34 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:937 | 26 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:938 | 25 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:939 | 29 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:940 | 29 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:941 | 31 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:942 | 32 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:943 | 32 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:944 | 31 happens. And 0 transactions are canceled. | And 0 totally failed.
In time slot:945 | 27 happens. A

KeyboardInterrupt: 